In [ ]:
# Salvar como install_from_pyproject.py e executar: python install_from_pyproject.py
import tomllib, subprocess, sys
with open("pyproject.toml","rb") as f:
    data = tomllib.load(f)
deps = data.get("project", {}).get("dependencies", [])
if not deps:
    print("Nenhuma dependency encontrada em pyproject.toml", file=sys.stderr); sys.exit(1)
for dep in deps:
    print("uv add", dep)
    subprocess.run(["uv", "add", dep], check=True)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Para o mapa
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

# Configurações gerais
plt.style.use('seaborn-v0_8')
sns.set_palette("hls", 8)
%matplotlib inline

In [ ]:
df = pd.read_csv('./data/winemag-data-130k-v2.csv')  # altera o nome se for diferente
df.head(200)

In [13]:
# Países da União Europeia (27 países em 2025)
eu_countries = ['Portugal', 'France', 'Italy', 'Spain', 'Germany', 'Austria', 
                'Greece', 'Hungary', 'Romania', 'Bulgaria', 'Croatia', 'Slovenia',
                'Slovakia', 'Czech Republic', 'Poland', 'Cyprus', 'Malta',
                'Belgium', 'Netherlands', 'Luxembourg', 'Ireland', 'Denmark',
                'Sweden', 'Finland', 'Estonia', 'Latvia', 'Lithuania']

# Filtrar apenas vinhos da UE
df_eu = df[df['country'].isin(eu_countries)].copy()

print(f"Total de reviews globais: {len(df)}")
print(f"Reviews de vinhos da UE: {len(df_eu)} ({len(df_eu)/len(df)*100:.1f}%)")

Total de reviews globais: 129971
Reviews de vinhos da UE: 60542 (46.6%)


In [ ]:
# Remover linhas sem país ou sem pontos
df_eu = df_eu.dropna(subset=['country', 'points'])

# Preencher preço missing com mediana por país (melhor que remover)
df_eu['price'] = df_eu.groupby('country')['price'].transform(lambda x: x.fillna(x.median()))
df_eu['price'].fillna(df_eu['price'].median(), inplace=True)

# Criar coluna qualidade/preço
df_eu['points_per_euro'] = df_eu['points'] / df_eu['price']

In [ ]:
top_countries = df_eu.groupby('country')['points'].mean().sort_values(ascending=False).round(2)

plt.figure(figsize=(12,7))
ax = top_countries.plot(kind='bar', color=sns.color_palette("viridis", len(top_countries)))
plt.title('Pontuação Média dos Vinhos por País da UE (Top 10)', fontsize=16, fontweight='bold')
plt.ylabel('Pontuação Média (0-100)')
plt.xlabel('País')
plt.xticks(rotation=45)
for i, v in enumerate(top_countries.head(10)):
    ax.text(i, v + 0.5, str(v), ha='center', fontweight='bold')
plt.tight_layout()
plt.show()

top_countries.head(10)

In [ ]:
quality_price = df_eu.groupby('country')['points_per_euro'].mean().sort_values(ascending=False)

plt.figure(figsize=(12,7))
ax = quality_price.head(10).plot(kind='bar', color=sns.color_palette("magma", 10))
plt.title('Melhor Relação Qualidade/Preço na UE (Pontos por Euro)', fontsize=16, fontweight='bold')
plt.ylabel('Pontos por Euro')
plt.xlabel('País')
plt.xticks(rotation=45)
for i, v in enumerate(quality_price.head(10)):
    ax.text(i, v + 0.01, f"{v:.3f}", ha='center', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
top5_countries = top_countries.head(5).index.tolist()

plt.figure(figsize=(12,7))
sns.boxplot(data=df_eu[df_eu['country'].isin(top5_countries)], x='country', y='points', palette="Set2")
plt.title('Distribuição da Pontuação - Top 5 Países da UE', fontsize=16, fontweight='bold')
plt.ylabel('Pontuação')
plt.xlabel('País')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df_eu[['points', 'price']].corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Correlação entre Preço e Pontuação (UE)')
plt.show()

In [ ]:
# Centro da Europa
europe_map = folium.Map(location=[54, 15], zoom_start=4, tiles='CartoDB positron')

# Média de pontos por país
country_points = df_eu.groupby('country')['points'].mean().round(1)

# Coordenadas aproximadas dos países (podes melhorar se quiseres)
coordinates = {
    'Portugal': [39.5, -8.2], 'Spain': [40.4, -3.7], 'France': [46.2, 2.2],
    'Italy': [42.5, 12.1], 'Germany': [51.1, 10.4], 'Austria': [47.5, 13.2],
    'Hungary': [47.1, 19.5], 'Romania': [45.9, 24.9], 'Greece': [39.0, 21.8],
    'Bulgaria': [42.7, 25.4], 'Croatia': [45.1, 15.2]
}

for country, coord in coordinates.items():
    if country in country_points.index:
        points = country_points[country]
        folium.CircleMarker(
            location=coord,
            radius=points/3,
            popup=f"<b>{country}</b><br>Pontuação média: {points}",
            color='crimson',
            fill=True,
            fill_color='crimson',
            fill_opacity=0.7
        ).add_to(europe_map)

europe_map.save('mapa_melhores_vinhos_UE.html')
europe_map

In [ ]:
top10_vinhos = df_eu.nlargest(10, 'points')[['title', 'country', 'winery', 'points', 'price', 'variety']]
top10_vinhos.index = range(1,11)
top10_vinhos

In [ ]:
# Para ver a app interativa, executa no terminal:
# streamlit run app.py

import os
if os.path.exists("app.py"):
    print("App Streamlit pronta! Abre o terminal e escreve:")
    print("streamlit run app.py")
else:
    print("Cria o ficheiro app.py com o código Streamlit para teres a versão interativa!")